In [1]:
import mlflow

In [2]:
import numpy as np
import glob
import os
import keras
from keras.utils import to_categorical

# Asumiendo que los archivos son .txt y que el directorio está en el path actual
file_paths = glob.glob("./UCIHARDataset/train/InertialSignals/*.txt")

# Lista para almacenar los datos de cada archivo
data_list = []

for file in file_paths:
    # Carga los datos del archivo en un array de numpy
    data = np.loadtxt(file)
    data_list.append(data)
    

# Convierte la lista de arrays en un solo array de numpy.
# Nota: Esto funcionará si todos los arrays tienen la misma forma.
data_array = np.array(data_list)
x_train = data_array.transpose(1, 2, 0)

file_paths = glob.glob("./UCIHARDataset/test/InertialSignals/*.txt")
# Lista para almacenar los datos de cada archivo
data_list = []

for file in file_paths:
    # Carga los datos del archivo en un array de numpy
    data = np.loadtxt(file)
    data_list.append(data)
    

# Convierte la lista de arrays en un solo array de numpy.
# Nota: Esto funcionará si todos los arrays tienen la misma forma.
data_array = np.array(data_list)
x_test = data_array.transpose(1, 2, 0)

y_train = np.loadtxt("./UCIHARDataset/train/y_train.txt")
y_test = np.loadtxt("./UCIHARDataset/test/y_test.txt")  


print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

y_train = to_categorical(y_train - 1)
y_test = to_categorical(y_test - 1)

print(y_train.shape)
print(y_test.shape)

(7352, 128, 9)
(2947, 128, 9)
(7352,)
(2947,)
(7352, 6)
(2947, 6)


In [3]:
# Normalize x_train and x_test using the mean and standard deviation from x_train
x_train_mean = x_train.mean()
x_train_std = x_train.std()

x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

print("x_train normalized: ", x_train.shape)
print("x_test normalized: ", x_test.shape)

x_train normalized:  (7352, 128, 9)
x_test normalized:  (2947, 128, 9)


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Input(shape=(128,9)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("Ejemplo con MLflow")



run = mlflow.start_run()
history = model.fit(x_train, y_train, epochs=4, batch_size=64, validation_data=(x_test, y_test),callbacks=[mlflow.keras.MlflowCallback(run)])
mlflow.end_run()

Epoch 1/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.9564 - loss: 0.1035 - val_accuracy: 0.9080 - val_loss: 0.3758
Epoch 2/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 63ms/step - accuracy: 0.9539 - loss: 0.1015 - val_accuracy: 0.9186 - val_loss: 0.2947
Epoch 3/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 7s 61ms/step - accuracy: 0.9549 - loss: 0.1008 - val_accuracy: 0.9186 - val_loss: 0.3100
Epoch 4/4
115/115 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.9633 - loss: 0.0899 - val_accuracy: 0.9179 - val_loss: 0.3771
🏃 View run adventurous-loon-473 at: http://127.0.0.1:8080/#/experiments/234596372402911890/runs/2818641bc06c44faadd5aeb5cd8b9d05
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/234596372402911890
